In [1]:
import numpy as np
import pandas as pd
import random, string
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [2]:
!ls

gender_submission.csv  test.csv       titanic.py
predictions.csv        Titanic.ipynb  train.csv


In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

#train_data, test_data = train_test_split(train_data, test_size=0.3)

train_data=train_data.set_index(train_data['PassengerId'])
test_data=test_data.set_index(test_data['PassengerId'])

train_data[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,,
1,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
mean = None
def prepare_both(X):
    X['Sex_i'] = X['Sex'].apply(lambda a: 1 if a == 'female' else 0)
    X['Age'] = X['Age'].fillna(29) # 29 is ~mean
    X['Embarked'] = X['Embarked'].fillna('A')
    X['Level'] = str(X['Cabin'].fillna('Z'))[0]
    
    s=['Sex_i', 'Age', 'SibSp', 'Parch']
    return pd.DataFrame(X[s], index=X.index)
        
def prepare_train(X):
    train_X = prepare_both(X)
    transformers=[]
    
    enc1 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(
            data=enc1.fit_transform(X[['Pclass']]).toarray(), index=X.index, 
            columns=enc1.get_feature_names()),
    )
    transformers.append(enc1)

    enc2 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(data=enc2.fit_transform(X[['Embarked']]).toarray(), index=X.index, 
                     columns=enc2.get_feature_names())
    )
    transformers.append(enc2)
    
    enc3 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(data=enc3.fit_transform(X[['Level']]).toarray(), index=X.index, 
                     columns=enc3.get_feature_names())
    )
    transformers.append(enc3)

    return train_X, transformers

def prepare_test(X, transformers):
    test_X = prepare_both(X)
    
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[0].transform(X[['Pclass']]).toarray(), index=X.index,
                     columns=transformers[0].get_feature_names())
    )
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[1].transform(X[['Embarked']]).toarray(), index=X.index,
                     columns=transformers[1].get_feature_names())
    )
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[2].transform(X[['Level']]).toarray(), index=X.index,
                     columns=transformers[2].get_feature_names())
    )
    return test_X

X_train, transformers = prepare_train(train_data)
X_test = prepare_test(test_data, transformers)

Y_train = train_data['Survived']
#Y_test = test_data['Survived']

X_train

,Sex_i,Age,SibSp,Parch,x0_1,x0_2,x0_3,x0_A,x0_C,x0_Q,x0_S,x0_P
PassengerId,,,,,,,,,,,,
1,0,22.0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
2,1,38.0,1,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1,26.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,1,35.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
5,0,35.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
888,1,19.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
889,1,29.0,1,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
#import warnings
#warnings.filterwarnings("ignore")

In [6]:
def train(X_train, Y_train, design):
    model = MLPClassifier(solver='sgd', hidden_layer_sizes=design, max_iter=1000, #verbose=10,
                          learning_rate='adaptive', learning_rate_init=0.005)
    model.fit(X_train, Y_train)
    return model 

model = train(X_train, Y_train, design=(50, 50, 50))
train_acc=model.score(X_train, Y_train)
print('train_acc={}, '.format(str(train_acc)))

#test_acc=model.score(X_test, Y_test)
#print('test_acc={}, '.format(str(test_acc)))

train_acc=0.8249158249158249, 


In [7]:
test_data = pd.read_csv('test.csv')
test_data=test_data.set_index(test_data['PassengerId'])
X_test = prepare_test(test_data, transformers)
X_test=X_test.join(pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['Survived']))
X_test[['Survived']].to_csv('predictions.csv')